In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Reshape, Dropout, Dense, Flatten, BatchNormalization, Conv2D, MaxPooling2D,BatchNormalization
from tensorflow.keras.optimizers import RMSprop
from PIL import Image 
from tensorflow.keras.optimizers import Adam, SGD, RMSprop

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/sudoku/sudoku.csv


In [2]:
df = pd.read_csv('../input/sudoku/sudoku.csv')
df.head()

,quizzes,solutions
0,0043002090050090010700600430060020871900074000...,8643712593258497619712658434361925871986574322...
1,0401000501070039605200080000000000170009068008...,3461792581875239645296483719658324174729168358...
2,6001203840084590720000060050002640300700800069...,6951273841384596727248369158512647392739815469...
3,4972000001004000050000160986203000403009000000...,4972583161864397252537164986293815473759641828...
4,0059103080094030600275001000300002010008200070...,4659123781894735623275681497386452919548216372...


In [3]:
X = np.array(df.quizzes.map(lambda x: list(map(int, x))).to_list())
Y = np.array(df.solutions.map(lambda x: list(map(int, x))).to_list())

In [4]:
X = X.reshape(-1, 9, 9, 1)
Y = Y.reshape(-1, 9, 9) - 1

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [6]:
X_train.shape, X_val.shape

((640000, 9, 9, 1), (160000, 9, 9, 1))

In [7]:
model = Sequential()
model.add(Conv2D(64, 3, activation='relu', padding='same', input_shape=(9,9,1)))
model.add(BatchNormalization())
model.add(Conv2D(64, 3, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, 3, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, 3, activation='relu', padding = 'same'))
model.add(BatchNormalization())
model.add(Conv2D(256, 3, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(256, 3, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(512, 3, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(9, 1, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(81*9))
model.add(Reshape((9, 9, 9)))
model.add(Activation('softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
                optimizer=Adam(
                learning_rate=0.001
    ),
    metrics=['accuracy'])
model.summary()
# history = model.fit(X_train, y_train, batch_size = 32, epochs = 200,validation_data=(X_test, y_test))

2022-10-06 08:17:38.456882: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 9, 9, 64)          640       
_________________________________________________________________
batch_normalization (BatchNo (None, 9, 9, 64)          256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 9, 9, 64)          36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 9, 9, 64)          256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 128)         73856     
_________________________________________________________________
batch_normalization_2 (Batch (None, 9, 9, 128)         512       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 9, 9, 128)         1

In [ ]:
history = model.fit(X_train, y_train, batch_size = 32, epochs = 200,validation_data=(X_test, y_test))

2022-10-06 08:17:39.739028: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/200
10812/20000 [===============>..............] - ETA: 38:15 - loss: 0.7350 - accuracy: 0.7465

In [ ]:
fig, ax = plt.subplots(1, 2)
fig.set_size_inches(12,6)
e = [i for i in range(1, 201)]
ax[0].plot(e, history.history['loss'], label='train loss')
ax[0].plot(e, history.history['val_loss'], label='val loss')
ax[0].legend()
ax[0].set_xlabel("Epochs")
ax[0].set_ylabel("loss")

ax[1].plot(e, history.history['accuracy'], label='accuracy')
ax[1].plot(e, history.history['val_accuracy'], label='val_accuracy')
ax[1].legend()
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Accuracy')
plt.show()

In [ ]:
model.save('sudoku.h5')

In [ ]:
inp = X_train[0].reshape((9, 9)).astype(int)
inp

In [ ]:
model.predict(inp.reshape(1, 9, 9, 1)).argmax(-1).squeeze()+1